INSTALAÇÃO DE BIBLIOTECAS NECESSÁRIAS


In [ ]:
!pip install xlwt
!pip install basedosdados
!pip install pandas
!pip install tabulate
!pip install os
!pip install openpyxl

##### Importando bibliotecas que vou usar 

In [16]:
import basedosdados as bd
import pandas as pd
from unidecode import unidecode
from tabulate import tabulate

## **Preços de Combustíveis - Dados históricos**
#### **Organização**:    Agência Nacional do Petróleo, Gás Natural e Biocombustíveis (ANP)
##### **Cobertura temporal** : 2020 - 2021 (Mês de janeiro)

In [ ]:


# Definir a query SQL para buscar apenas os dados de janeiro de 2021
sql_query = '''
SELECT *
FROM `basedosdados.br_anp_precos_combustiveis.microdados`
WHERE (EXTRACT(YEAR FROM data_coleta) = 2020 OR EXTRACT(YEAR FROM data_coleta) = 2021)
  AND EXTRACT(MONTH FROM data_coleta) = 1
'''

# Carregar apenas os dados filtrados para janeiro de 2021
df = bd.read_sql(sql_query, billing_project_id="leafy-ripsaw-382815")


#### Após fazer a query no banco de dados, salvei a pesquisa em um arquivo excel para facilitar as proximas pesquisas por causa da internet


In [ ]:
df.to_excel('dfcombustiveis.xlsx', index=False)


#### Lendo arquivo excel com a biblioteca pandas.


In [2]:
dfcombustiveis = pd.read_excel('dfcombustiveis.xlsx')

#### Filtrando dataFrame com colunas adequadas ao projeto, removendo colunas que não irei utilizar.


In [3]:
dfcombustiveis = dfcombustiveis.drop(['nome_estabelecimento','id_municipio','cep_revenda','cnpj_revenda','bairro_revenda','endereco_revenda','preco_compra','data_coleta'],axis=1)



#### convertendo todas as colunas para letras minúsculas, para facilitar a busca


In [4]:
dfcombustiveisLower = dfcombustiveis.applymap(lambda s: s.lower() if isinstance(s, str) else s)


#### Função para remover todos os acentos de todas as palavras nas colunas do dataframe


In [5]:
def remover_acentos(texto):
    if texto is not None:
        return unidecode(texto)
    else:
        return texto

#### remover acentos de todas as palavras das colunas do dataframe


In [6]:
colunas_selecionadas = ['bandeira_revenda', 'produto']
dfcombustiveisLower = dfcombustiveisLower.copy()
dfcombustiveisLower[colunas_selecionadas] = dfcombustiveisLower[colunas_selecionadas].apply(lambda col: col.map(remover_acentos) if col.dtype == object and not issubclass(col.dtype.type, pd.core.dtypes.dtypes.DatetimeTZDtype) else col)


## **Dados filtrados que estão dentro do dataframe escolhido**
#### Todos os dados abaixo podem ser pesquisados no DataFrame.

#### **Anos:**  2020,2021

#### **UF:**  rs, sp, rj, mg, am, ce, df, se, ro, pb, pe, ba, pa, al, ac, es, pr, rn, pi, to, ms, mt, go, rr, sc, ap, ma

#### **Revendedoras:**  branca, petrobras, distribuidora, s.a., raizen, ipiranga, nacional, gas, butano, copagaz, liquigas, ultragaz, supergasbras, energia, consigaz, rodoil, servgas, alesat, sabba, equador, rzd, distribuidora, bahiana, setta, distribuidora, sp, dislub, amazongas, fogas, temape, atlantica, larco, minasgas, federal, simarelli, sul, combustiveis, petrox, distribuidora, stang, tobras, charrua, raizen, mime, small, taurus, royal, fic, idaza, uni, ciapetro, pelikano, potencial, ngc, distribuidora, total, brasil, tdc, distribuidora, maxsul, americanoil, rejaile, rio, branco, petrobahia, torrao, petroserra, fan, dibrape, masut, distribuidora, podium, hora, watt, ruff, walendowsky, montepetro, d`mais, gran, petro, supergasbras, dinamo, rm, petroleo, acol

#### **Produtos:**  diesel, etanol, gasolina, diesel, s10, gasolina, aditivada, gnv, glp

#### **Unidade de medida:**  R$/litro, R$/m3, R$/13kg

In [17]:
print('Digite o número da opção que você quer filtrar:\n')
print('sigla(UF): 1\nAno: 2\nRevenda: 3\nProduto: 4')

while True:
  

    opcao = input("Qual número da opção você deseja (1, 2, 3 ou 4)? ")

    if opcao.isdigit():
        opcao = int(opcao)

        if opcao in [1, 2, 3, 4]:
            break

    print("Valor inválido. Tente novamente.")

if opcao == 1:
  siglas = dfcombustiveisLower['sigla_uf'].unique()
  siglas = siglas.tolist()
  sigla = input(f"Digite uma dessas siglas UF: {siglas}").lower()
  while sigla not in siglas:
    print("Sigla inválida. Tente novamente.")
    sigla = input(f"Digite uma dessas siglas UF: {siglas}").lower()
  dataframeFiltrado = dfcombustiveisLower[dfcombustiveisLower['sigla_uf'] == sigla]
  dataframeList = dataframeFiltrado.head().values.tolist()
  print(f"\n DataFrame - filtrado por: {sigla}\n")
  print(tabulate(dataframeList, headers=dfcombustiveis.columns, tablefmt='grid'))
elif opcao == 2:
    anos = dfcombustiveisLower['ano'].unique()
    listaAnos = anos.tolist()
    while True:
        try:
            ano = int(input(f"Digite qual ano você quer procurar: {listaAnos}"))
            if ano in listaAnos:
                break
            else:
                print("Ano inválido. Tente novamente.")
        except ValueError:
            print("Ano inválido. Digite um número inteiro.")
    dataframeFiltrado = dfcombustiveisLower[dfcombustiveisLower['ano'] == ano]
    dataframeList = dataframeFiltrado.head().values.tolist()
    print(f"\n DataFrame - filtrado por: {ano}\n")
    print(tabulate(dataframeList, headers=dfcombustiveis.columns, tablefmt='grid'))
elif opcao == 3:
  revendas = dfcombustiveisLower['bandeira_revenda'].unique()
  revendas = revendas.tolist()
  revenda = input(f"Digite qual dessas revendedoras você quer procurar: {revendas}").lower()
  while revenda not in revendas:
    print("Revenda inválida. Tente novamente.")
    revenda = input(f"Digite qual dessas revendedoras você quer procurar: {revendas}").lower()
  dataframeFiltrado = dfcombustiveisLower[dfcombustiveisLower['bandeira_revenda'] == revenda]
  dataframeList = dataframeFiltrado.head().values.tolist()
  print(f"\n DataFrame - filtrado por: {revenda}\n")
  print(tabulate(dataframeList, headers=dfcombustiveis.columns, tablefmt='grid'))
elif opcao == 4:
  produtos = dfcombustiveisLower['produto'].unique()
  produtos = produtos.tolist()
  produto = input(f"Digite qual desses combustíveis você quer procurar: {produtos} ").lower()
  while produto not in produtos:
    print("Produto inválido. Tente novamente.")
    produto = input(f"Digite qual desses combustíveis você quer procurar: {produtos} ").lower()
  dataframeFiltrado = dfcombustiveisLower[dfcombustiveisLower['produto'] == produto]
  dataframeList = dataframeFiltrado.head().values.tolist()
  print(f"\n DataFrame - filtrado por: {produto}\n")
  print(tabulate(dataframeList, headers=dfcombustiveis.columns, tablefmt='grid'))
elif opcao == 0:
  print('Você saiu do programa, obrigado!!')
  

Digite o número da opção que você quer filtrar:

sigla(UF): 1
Ano: 2
Revenda: 3
Produto: 4
Qual número da opção você deseja (1, 2, 3 ou 4)? ano
Valor inválido. Tente novamente.
Qual número da opção você deseja (1, 2, 3 ou 4)? 2
Digite qual ano você quer procurar: [2021, 2020]2021

 DataFrame - filtrado por: 2021

+-------+------------+------------------------------+------------+------------------+---------------+
|   ano | sigla_uf   | bandeira_revenda             | produto    | unidade_medida   |   preco_venda |
+=======+============+==============================+============+==================+===============+
|  2021 | rs         | branca                       | diesel     | r$/litro         |         3.499 |
+-------+------------+------------------------------+------------+------------------+---------------+
|  2021 | sp         | petrobras distribuidora s.a. | etanol     | r$/litro         |         2.999 |
+-------+------------+------------------------------+------------+-------

#### DataFrame que foi filtrado:

In [18]:
dataframeFiltrado


,ano,sigla_uf,bandeira_revenda,produto,unidade_medida,preco_venda
0,2021,rs,branca,diesel,r$/litro,3.499
1,2021,sp,petrobras distribuidora s.a.,etanol,r$/litro,2.999
2,2021,sp,petrobras distribuidora s.a.,gasolina,r$/litro,4.279
3,2021,sp,branca,diesel s10,r$/litro,3.499
4,2021,sp,branca,diesel s10,r$/litro,3.459
...,...,...,...,...,...,...
325428,2021,mg,branca,etanol,r$/litro,3.699
325429,2021,mg,alesat,gasolina,r$/litro,4.856
325430,2021,sp,petrobras distribuidora s.a.,etanol,r$/litro,3.399
325431,2021,mg,branca,diesel,r$/litro,3.865


In [19]:
import os
import xlwt
from pandas.io.formats.excel import ExcelFormatter
while True:
  resposta = str(input("Você quer salvar o arquivo nos formatos xlsx e csv ? digite (s) para sim ou (n) para não, ou qualquer teclas para finalizar...")).lower()
  if resposta =='n':
    print("Obrigado pela sua participação, agradeço que você não usou memoria da maquina para salvar o arquivo...... kkkkk")
    break
  elif resposta == 's':
        diretorio_destino = str(input("Digite o nome da pasta que deseja salvar o arquivo ou digite download para salvar no diretorio padrão.")).lower()
        if diretorio_destino == 'download':
          print("Processando, aguarde......")   
          diretorio_raiz = os.path.expanduser("~")
          diretorio_destino = os.path.join(diretorio_raiz, 'Downloads')
        elif diretorio_destino != 'download':
          print("Processando, aguarde......")
          diretorio_raiz = os.path.expanduser("~")
          diretorio_destino = os.path.join(diretorio_raiz, diretorio_destino)
        if not os.path.exists(diretorio_destino):
          os.makedirs(diretorio_destino)
        caminho_csv = os.path.join(diretorio_destino, 'dataframeCombustivel.csv')
        dataframeFiltrado.to_csv(caminho_csv, index=False)
        caminho_xlsx = os.path.join(diretorio_destino, 'dataframeCombustivel.xlsx')
        dataframeFiltrado.to_excel(caminho_xlsx, index=False)
        print(f"Arquivo salvo com sucesso.....no diretório: {diretorio_destino}")
        break
  else:
    print("Opção inválida. Tente novamente.")


Você quer salvar o arquivo nos formatos xlsx e csv ? digite (s) para sim ou (n) para não, ou qualquer teclas para finalizar...n
Obrigado pela sua participação, agradeço que você não usou memoria da maquina para salvar o arquivo...... kkkkk
